In [1]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = "/"
ADMIN_URL = "http://data-agent:8021"

In [2]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                           webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

In [3]:
# Spins up server to respond to any webhook events from the agent
import asyncio
asyncio.get_event_loop().run_until_complete(agent_controller.listen_webhooks())
# await agent_controller.listen_webhooks()

In [4]:
# Controllers re-emit webhooks as events through PyPubSub
# Basic state management is handled through defaults but you can choose to pass in custom handlers
custom_handler = []
agent_controller.register_listeners(custom_handler, defaults=True)

In [5]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(alias="Will")

In [6]:
print(invite)

{'connection_id': '2c0fb7cb-e409-47e9-b84d-e016759644b6', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '6f3e6a4c-d7f6-4c5a-bd7b-83de3ce6452e', 'recipientKeys': ['8Pk27WDNHxYxP9t2AXwF5NZQWgBHughT7FCnaCfwNMcy'], 'label': 'Bob', 'serviceEndpoint': 'http://192.168.65.3:8020'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNmYzZTZhNGMtZDdmNi00YzVhLWJkN2ItODNkZTNjZTY0NTJlIiwgInJlY2lwaWVudEtleXMiOiBbIjhQazI3V0ROSHhZeFA5dDJBWHdGNU5aUVdnQkh1Z2hUN0ZDbmFDZndOTWN5Il0sICJsYWJlbCI6ICJCb2IiLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly8xOTIuMTY4LjY1LjM6ODAyMCJ9'}


In [7]:
data_connection_id = invite["connection_id"]
print(data_connection_id)

2c0fb7cb-e409-47e9-b84d-e016759644b6


In [ ]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(data_connection_id)
print("ACCEPT REQUEST")
print(connection)

In [ ]:
# Print Connections
connection = await agent_controller.connections.get_connection(data_connection_id)
print("DATA AGENT CONNECTION")
print(connection)

In [ ]:
# Do a trust ping to bring accepted request into "ACTIVE" state
while connection["state"] != "active":
    trust_ping = await agent_controller.connections.trust_ping(data_connection_id, "hello")
    print("TUST PING TO ACTIVATE CONNECTION - DATA -> RESEARCH")
    print(trust_ping)
    time.sleep(5)
    connection = await agent_controller.connections.get_connection(data_connection_id)

In [ ]:
# Print connection list
connection = await agent_controller.connections.get_connection(data_connection_id)
print("DATA AGENT CONNECTION")
print(connection)

In [ ]:
#send some basic messages
message = await agent_controller.connections.send_message(data_connection_id,"hello from data world!")
print("BASIC MESSAGE - DATA -> RESEARCH")
print(message)